In [ ]:
import pandas as pd
import numpy as np
import datetime
import time 
import os
import gc


%matplotlib inline
%load_ext autoreload
%autoreload 2

In [ ]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
'''/anaconda3/lib/python3.7/site-packages/numpy/core/fromnumeric.py:51: FutureWarning: Series.nonzero() is deprecated and will be removed in a future version.Use Series.to_numpy().nonzero() instead
  return getattr(obj, method)(*args, **kwds)'''

In [ ]:
data_path = './data/'
!ls -lthr data/

In [ ]:
%%time
# nrows = 10000
nrows = None
train = pd.read_csv(data_path+'train.csv', nrows=nrows)

In [ ]:
def pshape(df):
    print(f'df len: {df.shape[0]:,}')

In [ ]:
pshape(train)

In [ ]:
test = pd.read_csv(data_path+'test.csv', nrows=1000)
test.shape

In [ ]:
train.head(2)

In [ ]:
# test_click_session_ids = test[test.impressions.notnull()].session_id.unique()

In [ ]:
# ts = np.random.choice(test_click_session_ids, 1)
# test[test.session_id==ts[0]]

In [ ]:
# train_click_ids = train[train.impressions.notnull()].session_id.unique()

In [ ]:
# ts = np.random.choice(train_click_ids, 1)
# train[train.session_id==ts[0]]

In [ ]:
# train.groupby('session_id').apply(lambda x: x.iloc[-1]['reference']).isna().sum()

In [ ]:
# train.groupby('session_id').apply(lambda x: x.iloc[-1]['impressions']).isna().sum()

### clip sessions off to last click

In [ ]:
%%time
# get all rows upto the last clickout action (some rows has reference but it's not clickout action)
def up_to_last_click(grp):
    check = grp.action_type == 'clickout item'
    if check.sum() != 0:
        return grp.iloc[:np.argwhere(check)[-1][0]+1]
    else:
        return grp
        
train = train.groupby('session_id').apply(up_to_last_click).reset_index(drop=True)

In [ ]:
%%time
# get utc timestamp
train['ts'] = train['timestamp'].apply(lambda t: datetime.datetime.utcfromtimestamp(t))

In [ ]:
%%time
# only look at sessions with clickouts (for now)
# first filter out sessions that does not have a click-out
def check_clickout(grp):
     # sessions has clickouts
    has_clickout = 'clickout item' in grp['action_type'].unique()
    # last row has reference and it's not nan
    has_ref = ((grp['action_type'].iloc[-1] == 'clickout item') & 
               (grp.iloc[-1][['impressions', 'reference', 'prices']].isna().sum()==0))
    return has_clickout & has_ref
    
clicked = train.groupby('session_id').apply(check_clickout)
click_session_ids = clicked[clicked].index
# filter
train = train[train.session_id.isin(click_session_ids)].reset_index(drop=True)

In [ ]:
# %%time
# # only look at sessions with clickouts (for now)
# # first filter out sessions that does not have a click-out
# def check_clickout(grp):
#      # sessions has clickouts
#     has_clickout = 'clickout item' in grp['action_type'].unique()
#     # last row has reference and it's not nan
#     has_ref = ((grp['action_type'].iloc[-1] == 'clickout item') & 
#                (pd.notnull(grp.iloc[-1]['impressions'])) &
#                (pd.notnull(grp.iloc[-1]['reference'])) &
#                (pd.notnull(grp.iloc[-1]['prices'])))
#     return has_clickout & has_ref
    
# clicked = train.groupby('session_id').apply(check_clickout)
# click_session_ids = clicked[clicked].index
# # filter
# train = train[train.session_id.isin(click_session_ids)].reset_index(drop=True)

In [ ]:
pshape(train)

In [ ]:
# d = train.groupby('session_id')
# for i, j in d:
#     print(j.head())
#     print('='*20)
#      # sessions has clickouts
#     has_clickout = 'clickout item' in j['action_type'].unique()
#     print(has_clickout)
#     # last row has reference and it's not nan
#     has_ref = ((j['action_type'].iloc[-1] == 'clickout item') & 
#                (pd.notnull(j.iloc[-1]['impressions'])) &
#                (pd.notnull(j.iloc[-1]['reference'])))
#     print(has_ref)
#     break

### meta info

In [ ]:
%%time
meta = pd.read_csv(data_path+'item_metadata.csv', nrows=nrows)
meta['properties'] = meta['properties'].str.lower()

In [ ]:
%%time
# checkout properties of items
def get_property_count(row):
    return len(row.split('|'))

item_p_ctn = meta['properties'].apply(get_property_count)
item_p_ctn.describe()

In [ ]:
%%time
def get_property(row):
    return row.lower().split('|')
meta['ps'] = meta['properties'].apply(get_property)
# numer of properties
meta['nprop'] = meta.ps.str.len()
# star ratings
meta['star'] = meta.properties.str.extract('[\|](\d) star')
meta['star'] = meta['star'].astype(float)

In [ ]:
%%time
# create rating columns
ratings = ['good rating', 'satisfactory rating', 'excellent rating']
for r in ratings:
    meta[r.replace(' ', '_')] = meta.properties.str.findall(f'\|{r}').str.len()

In [ ]:
meta = (meta[['item_id', 'nprop', 'star', 'good_rating', 'satisfactory_rating', 'excellent_rating']]
        .set_index('item_id'))

### Create session features
---

In [ ]:
# %%time
# # find out if same customers have multiple sessions
# s = train.drop_duplicates(subset=['user_id', 'session_id'])
# s.session_id.duplicated().sum()
# # seems like not

In [ ]:
# train.current_filters[:1000].unique()

In [ ]:
%%time
def get_filters(x):
    if type(x) == str or type(x) == list:
        return x.split('|')
    else:
        return np.nan
def get_impressions(x):
    if type(x) == str:
        return x.split('|')
    else:
        return np.nan
    
    
train['filters'] = train.current_filters.str.split('|')
train['nfilters'] = train.filters.str.len()
train['imps_list'] = train.impressions.str.split('|')
nn_mask = train['imps_list'].notnull()
train.loc[nn_mask, 'imps_list'] = train.loc[nn_mask, 'imps_list'].apply(lambda x: [int(i) for i in x])
train['nimps'] = train.imps_list.str.len()

In [ ]:
# train.nimps.describe()

In [ ]:
# # no switch of devices during session
# (train.groupby('session_id')['device'].nunique()!=1).sum()

In [ ]:
# # differnet city during session
# (train.groupby('session_id')['city'].nunique()!=1).sum()

In [ ]:
%%time
# some custom funcs used in agggregation
def mean_dwell_time(x):
    if len(x) == 1:
        return 0
    else:
        return np.mean(np.diff(np.sort(x)))
    
def var_dwell_time(x):
    if len(x) == 1:
        return 0
    else:
        return np.var(np.diff(np.sort(x)))
    
def get_first(x):
    return x.iloc[0]

def get_last(x):
    return x.iloc[-1]

def n_clickouts(x):
    return (x=='clickout item').sum()

def click_rel_pos_avg(x):
    return np.mean(np.argwhere((x=='clickout item')))/len(x)

def ptp(x):
    return x.max() - x.min()


# define some aggs
# session_aggs = {'timestamp': [np.ptp, mean_dwell_time, var_dwell_time],
session_aggs = {'timestamp': [ptp, mean_dwell_time, var_dwell_time],
                'step': ['max'],
                'action_type': ['nunique', n_clickouts, click_rel_pos_avg],
                'city': ['nunique', get_first],
                'platform': [get_first],
                'device': [get_first],
                'nfilters': ['mean', 'max', 'min', get_last],
                'nimps': ['max']
               }

In [ ]:
session_grp = train.groupby('session_id')

In [ ]:
%%time
# import os.path
session_file = 'session_fts.csv'
if os.path.isfile(session_file):
    session_fts = pd.read_csv(session_file)
else:
    session_fts = session_grp.agg(session_aggs)
    session_fts.columns = ['_'.join(col).strip() for col in session_fts.columns.values]
    session_fts.to_csv(session_file)

In [ ]:
# session_fts.columns.values
session_fts.head()

In [ ]:
# train[train.session_id=='62991f7c78f27']

### Create clickout features
---

In [ ]:
import multiprocessing as mp
from functools import partial


def get_session_item_pairs(args):
    # grab the args
    gids, session_df, meta_df = args
    # selecting the assigned session ids and grouping on session level
    grps = (session_df[session_df['session_id'].isin(gids)]
            .reset_index(drop=True)
            .groupby('session_id'))
    
    # use apply to compute session level features
    session_compute_func = partial(compute_session_item_pair, meta_df=meta_df)
    session_features = grps.apply(session_compute_func)
    
    return session_features
        
    
# def compute_session_item_pair(session_df, g_id, buy_df):
def compute_session_item_pair(session_df, meta_df):
    sdf = session_df.copy()
#     with Timer('select_rows', profile) as t:
    last_row = sdf.iloc[-1]
    above = sdf.iloc[:-1]
#     with Timer('exclude_nans', profile) as t:
    # get previous appeard impressions
    prev = above[above['impressions'].notnull()]
#     with Timer('get_imp_list', profile) as t:
    prev_imps = prev['imps_list']
    unique_imps = [j for i in prev_imps for j in i]

#     with Timer('get_price', profile) as t:
    imp_l = last_row['imps_list']
#     try:
    prices = last_row['prices'].split('|')
#     except:
#         print(last_row['prices'], last_row)
    prices = [int(p) for p in prices]
    # whether the impression appeared before
    appeared = [int(i in unique_imps) for i in imp_l]
    # the location of the impression
    locs = list(range(len(imp_l)))

#     with Timer('create_df', profile) as t:
# build the df
    result = pd.DataFrame({'appeared': appeared, 'location': locs, 'price': prices}, index=imp_l)
    result.index.name = 'item_id'
#     with Timer('rel_price_rank', profile) as t:
    price_ind = np.argsort(result['price'].values) + 1
    result['rel_price_rank'] = price_ind/len(imp_l)
#         result['rel_price_rank'] = result[['location', 'price']].sort_values(by='price')['location']/len(imp_l)

#     with Timer('compute_mean_median', profile) as t:
    result['price_mean'] = np.mean(result['price'])
    result['price_median'] = np.median(result['price'])

#     with Timer('compute_diff', profile) as t: 
    result_price = result['price'].values
    result_price_mean = result['price_mean'].values 
    result_price_median = result['price_median'].values

    result['diff_mean'] = result_price - result_price_mean
    result['diff_median'] = result_price - result_price_median
    result['diff_mean_rel'] = (result_price - result_price_mean)/result_price
    result['diff_median_rel'] = (result_price - result_price_median)/result_price

#     with Timer('join', profile) as t:
    # fetch the meta data
    result = result.join(meta_df, on='item_id')
#     with Timer('create_mean_meta1', profile) as t:
    result['star_mean'] = np.mean(result['star'].values)
#     with Timer('create_mean_meta2', profile) as t:
    result['gr_mean'] = np.mean(result['good_rating'].values)
#     with Timer('create_mean_meta3', profile) as t:
    result['sr_mean'] = np.mean(result['satisfactory_rating'].values)
#     with Timer('create_mean_meta4', profile) as t:
    result['er_mean'] = np.mean(result['excellent_rating'].values)
#     with Timer('create_mean_meta5', profile) as t:
    result.reset_index(inplace=True)

#     with Timer('create target', profile) as t:
    # get target
    ref = int(last_row['reference'])
    result['target'] = (result['item_id'].values == ref).astype(int)
    return result


# 2) dwell time for each clickout
# 3) the rating and star and nproperty -> mean and median for 
#     the other impressions in list (or the quantile of these and prices)
# 4) location and relative location in the impressions list


def generate_session_item_pairs(sessions_df, meta_df, nprocs=None):
    t1 = time.time()
    if nprocs is None:
        nprocs = mp.cpu_count() - 1 
        print('Using {} cores'.format(nprocs))

    sids = sessions_df.session_id.unique()
    
    pairs = []
    # create iterator to pass in args
    def args_gen():
        for i in range(nprocs):
            yield (sids[range(i, len(sids), nprocs)], sessions_df, meta_df)
    
    # init multiprocessing pool
    pool = mp.Pool(nprocs)
    for pair in pool.map(get_session_item_pairs, args_gen()):
        pairs.append(pair)
    pool.close()
    pool.join()
    print('Done genearting, total time took: {0:.2f}mins'.format((time.time()-t1)/60))

    return pd.concat(pairs, axis=0)#, ignore_index=True)

In [ ]:
# %%time
# result_df = generate_session_item_pairs(train[train.session_id.isin(sids)], meta, nprocs=None)

In [ ]:
# %%time
# result_df = generate_session_item_pairs(train[:10000], meta, nprocs=None)

In [ ]:
%%time
result_df = generate_session_item_pairs(train, meta, nprocs=None)

In [ ]:
result_df.to_csv('./data/result.csv')

In [ ]:
result_df.shape

In [ ]:
meta.shape

In [ ]:
result_df.head()

In [ ]:
%%time
result_df = result_df.reset_index(level='session_id')

In [ ]:
%%time
# join on session features
result_df = result_df.set_index('session_id')

In [ ]:
%%time
final = result_df.join(session_fts)

to do:

1) check the time range of train vs test

In [ ]:
gc.collect()